In [1]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from flaml.automl import AutoML
from collections import Counter
# from lazyqsar.models import LazyXGBoostBinaryClassifier as LQ_XGB
# from lazyqsar.models import LazyZSRFBinaryClassifier as LQ_zsRF
import sys
import os
sys.path.insert(0, os.path.abspath("../../lazy-qsar/lazyqsar/models"))
from xgboost_binary_classifier import LazyXGBoostBinaryClassifier as LQ_XGB
from zsRF_binary_classifier import LazyZSRFBinaryClassifier as LQ_zsRF
import pandas as pd
import numpy as np
import collections
import joblib

In [ ]:
# Get all pathogens i.e. {pathogen}_{target}
PATHOGENS = sorted(os.listdir(os.path.join("..", "data")))[:1]

# Define some paths
PATH_TO_FEATURES = os.path.join("..", "output", "02_features")
PATH_TO_OUTPUT = os.path.join("..", "output", "03_baseline_models")

for pathogen in PATHOGENS:

    print(f"----------------------- PATHOGEN: {pathogen} ---------------------------")

    # Get list of tasks
    tasks = sorted(os.listdir(os.path.join("..", "data", pathogen)))

    # For each task
    for task in tasks:

        # # if task is not done yet
        # if os.path.exists(os.path.join(PATH_TO_OUTPUT, pathogen, task.replace(".csv", ""), "LQ_optuna_CV.csv")) == True:
        #     continue

        print(f"TASK: {task}")

        # Create output_dir
        output_dir = os.path.join(PATH_TO_OUTPUT, pathogen, task.replace(".csv", ""))
        os.makedirs(output_dir, exist_ok=True)


        # Get IK to MFP
        IKs = open(os.path.join(PATH_TO_FEATURES, pathogen, 'IKS.txt')).read().splitlines()
        MFPs = np.load(os.path.join(PATH_TO_FEATURES, pathogen, "X.npz"))['X']
        IK_TO_MFP = {i: j for i, j in zip(IKs, MFPs)}

        # Load data
        df = pd.read_csv(os.path.join("..", "data", pathogen, task))
        cols = df.columns.tolist()
        X, Y = [], []
        for ik, act in zip(df['inchikey'], df[cols[2]]):
            if ik in IK_TO_MFP:
                X.append(IK_TO_MFP[ik])
                Y.append(act)

        # To np.array
        X = np.array(X)
        Y = np.array(Y)

        # Cross-validations
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        aurocs_zsRF, aurocs_XGB = [], []
        for train_index, test_index in skf.split(X, Y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]

            # XGB
            model_XGB = LQ_XGB(reducer_method='pca', max_reducer_dim=100, base_num_splits=1, base_test_size=0.2)
            model_XGB.fit(X=X_train, y=y_train)
            fpr, tpr, _ = roc_curve(y_test, model_XGB.predict(X_test))
            auroc = auc(fpr, tpr)
            aurocs_XGB.append(auroc)

            # zsRF
            model_zsRF = LQ_zsRF(reducer_method='pca', max_reducer_dim=100, base_num_splits=1, base_test_size=0.2)
            model_zsRF.fit(X=X_train, y=y_train)
            fpr, tpr, _ = roc_curve(y_test, model_zsRF.predict(X_test))
            auroc = auc(fpr, tpr)
            aurocs_zsRF.append(auroc)

        # Save AUROC CVs zsRF
        with open(os.path.join(PATH_TO_OUTPUT, pathogen, task.replace(".csv", ""), "LQ_zsRF_CV.csv"), "w") as f:
            f.write(",".join([str(round(i, 4)) for i in aurocs_zsRF]))

        # Save AUROC CVs XGB
        with open(os.path.join(PATH_TO_OUTPUT, pathogen, task.replace(".csv", ""), "LQ_XGB_CV.csv"), "w") as f:
            f.write(",".join([str(round(i, 4)) for i in aurocs_XGB
                              ]))

----------------------- PATHOGEN: abaumannii_organism ---------------------------
TASK: 1_assay_CHEMBL4296188_Inhibition_percentage_activity_percentile_50_ORGANISM_2.csv
Total samples: 35812, positive samples: 7162, negative samples: 28650
Positive proportion: 0.20
Sampling rounds: 1, positive samples per round: 7162, negative samples per round: 28650
Desired positive proportion: 0.19998883055958896 Actual positive proportion:  0.19998883055958896


[I 2025-06-05 16:28:20,584] A new study created in memory with name: no-name-30f0a387-a214-494a-bead-b60db53286bc


Fitting model on 35812 samples, positive samples: 7162, negative samples: 28650, number of features 100
Fitting...


[I 2025-06-05 16:28:20,840] Trial 0 finished with value: 0.9145740699468645 and parameters: {'booster': 'gblinear', 'lambda': 0.0006155564318973012, 'alpha': 1.77071686435378e-07, 'subsample': 0.32479561626896214, 'colsample_bytree': 0.24646688973455957}. Best is trial 0 with value: 0.9145740699468645.
[I 2025-06-05 16:28:21,205] Trial 1 finished with value: 0.8760070831034614 and parameters: {'booster': 'gbtree', 'lambda': 1.4610865886287176e-08, 'alpha': 0.574485163632042, 'subsample': 0.8659541126403374, 'colsample_bytree': 0.36987128854262097, 'max_depth': 3, 'min_child_weight': 3, 'eta': 2.716051144654844e-06, 'gamma': 0.00015777981883364995, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.9145740699468645.
[I 2025-06-05 16:28:21,523] Trial 2 finished with value: 0.7783084827958285 and parameters: {'booster': 'gbtree', 'lambda': 8.528933855762793e-06, 'alpha': 4.452048365748842e-05, 'subsample': 0.8281407691144109, 'colsample_bytree': 0.3597390257266878, 'max_depth': 7,

Best AUROC: 0.9306
Model fitted.
Total samples: 35812, positive samples: 7162, negative samples: 28650
Positive proportion: 0.20
Sampling rounds: 1, positive samples per round: 7162, negative samples per round: 28650
Desired positive proportion: 0.19998883055958896 Actual positive proportion:  0.19998883055958896
Fitting model on 35812 samples, positive samples: 7162, negative samples: 28650, number of features 100
Model fitted.
Total samples: 35813, positive samples: 7163, negative samples: 28650
Positive proportion: 0.20
Sampling rounds: 1, positive samples per round: 7163, negative samples per round: 28650
Desired positive proportion: 0.20001116912852876 Actual positive proportion:  0.20001116912852876


...

[I 2025-06-05 17:07:48,802] A new study created in memory with name: no-name-fdb191ad-7c19-4983-99df-1e84874b491d


Fitting model on 344 samples, positive samples: 69, negative samples: 275, number of features 100
Fitting...


[I 2025-06-05 17:07:49,097] Trial 0 finished with value: 0.7376623376623377 and parameters: {'booster': 'gblinear', 'lambda': 0.0006155564318973012, 'alpha': 1.77071686435378e-07, 'subsample': 0.32479561626896214, 'colsample_bytree': 0.24646688973455957}. Best is trial 0 with value: 0.7376623376623377.
[I 2025-06-05 17:07:49,106] Trial 1 finished with value: 0.6616883116883117 and parameters: {'booster': 'gbtree', 'lambda': 1.4610865886287176e-08, 'alpha': 0.574485163632042, 'subsample': 0.8659541126403374, 'colsample_bytree': 0.36987128854262097, 'max_depth': 3, 'min_child_weight': 3, 'eta': 2.716051144654844e-06, 'gamma': 0.00015777981883364995, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.7376623376623377.
[I 2025-06-05 17:07:49,118] Trial 2 finished with value: 0.5 and parameters: {'booster': 'gbtree', 'lambda': 8.528933855762793e-06, 'alpha': 4.452048365748842e-05, 'subsample': 0.8281407691144109, 'colsample_bytree': 0.3597390257266878, 'max_depth': 7, 'min_child_wei

Best AUROC: 0.8805
Model fitted.
Total samples: 344, positive samples: 69, negative samples: 275
Positive proportion: 0.20
Sampling rounds: 1, positive samples per round: 69, negative samples per round: 275
Desired positive proportion: 0.2005813953488372 Actual positive proportion:  0.2005813953488372
Fitting model on 344 samples, positive samples: 69, negative samples: 275, number of features 100
Model fitted.


: 